Async/Await and Non-Blocking Execution
=======================================

Dask integrates natively with concurrent applications using the [Tornado](https://www.tornadoweb.org/en/stable/) or [Asyncio](https://docs.python.org/3/library/asyncio.html) frameworks, and can make use of Python's `async` and `await` keywords.

This example shows a small example how how to start up a Dask Client in asynchronous mode.

The `asynchronous=True` parameter
---------------------------------

Dask LocalCluster and Client objects can operate in async-await mode if you pass the `asynchronous=True` parameter.

In [1]:
from dask.distributed import Client
client = await Client(asynchronous=True)

In [2]:
def inc(x: int) -> int:
    return x + 1

future = client.submit(inc, 10)
future

<Future: pending, key: inc-776ab41f8b3f7db739665decc4422ee9>

In [3]:
await future

11

Collections
-----------

Note that blocking operations like the `.compute()` method aren't ok to use in asynchronous mode.  Instead you'll have to use the `Client.compute` method.

In [4]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [5]:
df = df.persist()             # persist is non-blocking, so it's ok

In [6]:
total = df[['x', 'y']].sum()  # lazy computations are also ok

In [7]:
# total.compute()             # but compute is bad, because compute blocks until done

In [8]:
future = client.compute(total)
future

<Future: pending, key: finalize-d0836e36a25fd21da2ea9accd5d4a6e8>

In [9]:
await future

x    1723.547025
y     458.015485
dtype: float64

Within a script
---------------

Running async/await code in Jupyter is a bit atypical.  Jupyter already has an event loop running, so it's easy to use async/await syntax directly within it.  In a normal Python script this won't be the case.  Here is an example script that should run within a normal Python interpreter or as a script.

```python
import asyncio
from dask.distributed import Client


def inc(x: int) -> int:
    return x + 1


async def f():
    async with Client(asynchronous=True) as client:
        future = client.submit(inc, 10)
        result = await future
        print(result)


if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(f())
```